In [1]:
load("../Imports/GeneralMethods.sage", '../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
Q.<rad2_minus_i> = NumberField((sqrt(2)-I).minpoly('xx'))

In [3]:
ii = ((rad2_minus_i)^3+rad2_minus_i)/(-6)

In [4]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [5]:
# Define cubic SE1' and calculate the admissible projectivities.

In [6]:
SE1_1 = general_cubic.subs({b:-(c*c+e*f)/c}).subs({e:(c*(2*ii-1)*(5*c-(4*ii-3)*f))/(5*(c-f)), d:((2*ii+1)*(5*c^2-5*f^2-(4*ii+8)*c*f))/(5*(f-c))})
adm_SE1_1 = SE1_1.find_admissible_projectivities()

In [7]:
# Find condition associated to L-set ('G4', 'E1', 'F15', 'E5', 'F12').

In [8]:
projectivity = [proj for proj in adm_SE1_1 if proj[1] == ('G4', 'E1', 'F15', 'E5', 'F12')]
conds = SE1_1.find_conditions_for_subfamilies(projectivity)
conds

[([Ideal (3*c + (rad2_minus_i)*f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in rad2_minus_i with defining polynomial xx^4 - 2*xx^2 + 9,
   Ideal (9*c + (rad2_minus_i^3 - 2*rad2_minus_i)*f) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in rad2_minus_i with defining polynomial xx^4 - 2*xx^2 + 9],
  ('G4', 'E1', 'F15', 'E5', 'F12'))]

In [9]:
# Extract substitution to obtain cubic surfaces with bigger simmetry group.

In [10]:
prim_decs = [el[0] for el in conds]
ideals = list(set([ideal for prim_dec in prim_decs for ideal in prim_dec]))
sosts = [solve_linear_system(ideal.gens(), [c], [f]) for ideal in ideals]
["Substitution: (c, f) = " + str([el/f for el in sost]) for sost in sosts]

['Substitution: (c, f) = [(-rad2_minus_i), 3]',
 'Substitution: (c, f) = [(-rad2_minus_i^3 + 2*rad2_minus_i), 9]']

In [11]:
# Define cubic SE1'', find and classify the lines and compute the 45 tritangent planes

In [12]:
SE1_2 = SE1_1.subs({c:-rad2_minus_i, f:3})

In [13]:
# Find simmetries

In [14]:
%time adm_SE1_2 = SE1_2.find_admissible_projectivities()
len(adm_SE1_2)

CPU times: user 4 s, sys: 191 ms, total: 4.19 s
Wall time: 23.5 s


576

In [15]:
%time simm_SE1_2 = SE1_2.find_simmetries(adm_SE1_2)
len(simm_SE1_2)

CPU times: user 2.6 s, sys: 158 ms, total: 2.76 s
Wall time: 3.1 s


8

In [16]:
# Study how the simmetries permute the Eckardt points

In [17]:
Eck_perms = [Permutation(SE1_2.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE1_2]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(1, '1')

In [18]:
### geometric interpretation of the group of symmetries

In [19]:
%%time
## Action of the stabilizer of SE1'' on the 27 lines:
Ga = perm_group_lines(SE1_2, simm_SE1_2)

## Action of the stabilizer of SE1'' on the 45 tritangent planes:
Gb = perm_group_planes(SE1_2, simm_SE1_2)

CPU times: user 4.78 s, sys: 23 ms, total: 4.81 s
Wall time: 4.81 s


In [20]:
print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
C8

Structure of the group Gb:
C8


In [21]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [22]:
# The only lines fixed is F14
for orb in OGa:
    if len(orb) == 1:
        print(simbolic_line_from_index(orb[0]))

F14


In [24]:
# The only tritangent plane fixed is tau_3
for orb in OGb:
    if len(orb) == 1:
        fixed_plane = simbolic_plane_from_index(orb[0])
        print(fixed_plane, tritangent_plane_from_name(fixed_plane))

t3 ['E1', 'G4', 'F14']
